In [1]:
import pandas as pd
import emoji
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.model_selection import train_test_split
from keras.preprocessing import sequence, text
from keras.models import load_model

In [2]:
#data = pd.read_csv("final_data.csv")
sent_to_id  = {"empty":0, "sadness":1,"excited":2,"fear":3,
                        "surprise":4,"disgust":5,"happiness":6,"anger":7}
contractions = pd.read_csv("contractions.csv")
cont_dic = dict(zip(contractions.Contraction, contractions.Meaning))

In [3]:
misspell_data = pd.read_csv("aspell.txt",sep=":",names=["correction","misspell"])
misspell_data.misspell = misspell_data.misspell.str.strip()
misspell_data.misspell = misspell_data.misspell.str.split(" ")
misspell_data = misspell_data.explode("misspell").reset_index(drop=True)
misspell_data.drop_duplicates("misspell",inplace=True)
miss_corr = dict(zip(misspell_data.misspell, misspell_data.correction))

#Sample of the dict
#{v:miss_corr[v] for v in [list(miss_corr.keys())[k] for k in range(20)]}

In [4]:
def misspelled_correction(val):
    for x in val.split(): 
        if x in miss_corr.keys(): 
            val = val.replace(x, miss_corr[x]) 
    return val

In [5]:
def cont_to_meaning(val): 
  
    for x in val.split(): 
        if x in cont_dic.keys(): 
            val = val.replace(x, cont_dic[x]) 
    return val

In [6]:
def punctuation(val): 
  
    punctuations = '''()-[]{};:'"\,<>./@#$%^&_~'''
  
    for x in val.lower(): 
        if x in punctuations: 
            val = val.replace(x, " ") 
    return val


In [7]:
def clean_text(val):
    val = misspelled_correction(val)
    val = cont_to_meaning(val)
    #val = p.clean(val)
    val = ' '.join(punctuation(emoji.demojize(val)).split())
    
    return val

In [8]:
import pickle
with open('tokenizer.pickle', 'rb') as handle:
    token = pickle.load(handle)
batch_size = 32
model = load_model('my_model3.h5')

In [9]:
max_len = 160
def output2(text1):
    text = clean_text(text1)
    twt = token.texts_to_sequences([text1])
    twt = sequence.pad_sequences(twt, maxlen=max_len, dtype='int32')
    sentiment = model.predict(twt,batch_size=1,verbose = 2)
    result = pd.DataFrame([sent_to_id.keys(),sentiment[0]]).T
    result=result.dropna()
    result.columns = ["sentiment","percentage"]
    result=result.T
    result.columns = result.iloc[0]
    result.drop(['sentiment'],axis=0,inplace=True)
    result=result.iloc[:1]
    r = result.to_numpy().tolist()
    r=r[0]
    return r
    
#     new_header = result.iloc[0] #grab the first row for the header
#     result = result[1:] #take the data less the header row
#     result.columns = new_header
    #return r

In [10]:
output2('As early as I could remember myself, my memories have been colored with melancholy and sad feelings. Yes, I admit it: I am a person who loves being sad. As a child, I loved gloomy fairy tales; as a teenager, I loved tragic novels and films, which made my parents worry a lot. I realized that I did not meet their expectations: in their ideal world, a happy person must irradiate joy all the time. In my view, this state of constant life enjoyment looked slightly idiotic. And I kept asking myself if sadness is really so bad.')

1/1 - 0s


[0.010666491,
 0.5506197,
 0.0038057922,
 0.2927482,
 0.05176571,
 0.08408123,
 0.0043951804,
 0.0019175473]

In [ ]:
# file =pd.read_excel("Sample Data File.xlsx",sheet_name='Formulas')
# file.drop(file.iloc[:, 1:], inplace = True, axis = 1)
# file.dropna(inplace=True)
# file['new']=file['text'].apply(output2)
# file[['empty','sadness','excited','fear','surprise','disgust','happiness','anger']] = pd.DataFrame(file.new.tolist(),index= file.index)
# file.drop(columns={'new'},inplace=True)
# file

In [ ]:
# file[['empty','sadness','excited','fear','surprise','disgust','happiness','anger']] = pd.DataFrame(file.new.tolist(),index= file.index)
# file.drop(columns={'new'},inplace=True)
# file